In [27]:
EXPERIMENT_IDENTIFIER = "debug"

In [28]:
N = 5000

In [3]:
!ls ../data/nonethnic_dataset_100k.joblib -alh

-rw-rw-r-- 1 abhijit abhijit 212M আগস্ট     29 19:31 ../data/nonethnic_dataset_100k.joblib


In [22]:
import pandas as pd
import joblib
import random
random.seed(42)
nonethnic_articles = joblib.load("../data/nonethnic_dataset_100k.joblib")
articles = random.sample(nonethnic_articles, N)
print(len(articles))
articles[:2]

5000


['<p>ফকিরহাট উপজেলা স্বাস্থ্য ও পরিবার পরিকল্পনা কর্মকর্তা অসীম কুমার সমাদ্দার বলেন, খুলনায় চিকিৎসাধীন অবস্থায় পল্লিচিকিৎসক ও তাঁর ছেলের মৃত্যু হয়েছে। এ নিয়ে ফকিরহাট উপজেলায় মোট চারজনের মৃত্যু হলো।</p>',
 "ব্লু-হোয়েল শেষে অতঃপর চলছে মোমো আতঙ্ক। তবে মোমোর ছোবলের মাঝেই হাজির হলো আরেক নতুন মরণগেম। এ নতুন অনলাইন মরণগেমের নাম গ্র্যানি।এই গেমের খপ্পরে পড়ে অসুস্থ হয়ে পড়ছে অনেক কোমলমতি শিক্ষার্থী। বিশেষ করে ভারতে ফাঁদ পেতেছে এ মরণগেম। গ্র্যানির ফাঁদে পা রাখার জন্য অনেককেই এসএমএস বা অনলাইনে আমন্ত্রণ জানানো হয়েছে।ইতিমধ্যে গ্র্যানি নামক এই মোবাইল গেমটি খেলে অসুস্থ হয়ে পড়েছে তিন স্কুলছাত্র। খবর: জি নিউজের।ঘটনাটি ঘটেছে ভারতের জলপাইগুড়ির ময়নাগুড়িতে।খবরে প্রকাশ, অস্বাভাবিক আচরণ করছে গ্র্যানিগেমে আসক্ত ওইসব ছাত্ররা। বাড়ির লোকজনকে মারধর করছে তারা।আবার কখনো তারা নিজেই মরে যেতে চাইছে। 'আমি বাঁচতে চাই না, মরতে দাও' চিৎকার করে বাড়িকে মাথায় তুলছে তারা।পরিস্থিতি ভয়াবহ দেখে স্থানীয় থানার দ্বারস্থ হয়েছে তাদের পরিবার।গ্র্যানিগেম আসক্ত ওই তিন ছাত্রের দুজন দশম শ্রেণির, একজন একাদশ শ্রেণির ছাত্র।জানা গেছে, দিনক

In [5]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

### Loading model

In [6]:
pip install openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [8]:
from openai import OpenAI
import json

client = OpenAI()

def get_response(news_article):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": news_article}
        ]
    )
    return completion.choices[0].message.content

In [9]:
with open('prompt.txt', 'r') as file:
    prompt = file.read()
print(prompt)

You are a Bangla news article analyzer. Given an article, analyze the following aspects:

    1. Genre: Identify the genre(s) of the article. Choose from: Politics, Crime, International Affairs, Business & Economy, Sports, Culture & Entertainment, Jobs, Lifestyle, Health, Environment, Local news, Education, Technology, Religion, Science, Protests and Social Movements, Govt Actions.
    2. Journalism Style: Determine the journalism style used in the article. Choose from: Straight, Investigative, Commentary.
    3. indigenous People Representation:
        - Identify if indigenous people are featured in the article.
        - If yes, categorize their role as one of the following: Subject, Spokesperson, Expert or Commentator, Personal Experience, Popular Opinion, Eyewitness.
        - If indigenous people are only mentioned in passing, return false. Otherwise, return true.
    4. Sentiment: Assess the sentiment of the article towards indigenous people. Choose from: Positive, Negative, Neu

In [30]:
N

5000

In [35]:
from tqdm import tqdm
import pickle

# Define checkpoint file path
checkpoint_file = 'responses_checkpoint_nonethnic_v3.pkl'

# Load checkpoint if it exists
try:
    with open(checkpoint_file, 'rb') as f:
        responses = pickle.load(f)
        start_index = len(responses)
        tqdm.write(f"Resuming from checkpoint at index {start_index}")
except FileNotFoundError:
    responses = []
    start_index = 0

# Ensure that articles and N are properly defined
N = len(articles)  # Assuming articles is a list of text data

# Fetch responses with checkpoints
for i, article in tqdm(enumerate(articles[start_index:N]), total=N - start_index, desc="Processing articles"):
    # Ensure we're only processing from the checkpoint onward
    response = get_response(article[:10000])  # Slice article to handle length limit
    responses.append(response)
    
    # Save checkpoint every 100 responses
    if (i + start_index + 1) % 100 == 0:
        with open(checkpoint_file, 'wb') as f:
            pickle.dump(responses, f)
        tqdm.write(f"Checkpoint saved at index {i + start_index + 1}")

# Save final responses
with open(checkpoint_file, 'wb') as f:
    pickle.dump(responses, f)
tqdm.write("Processing complete and final checkpoint saved.")

Resuming from checkpoint at index 2600


Processing articles:   4%|███▋                                                                                     | 100/2400 [02:36<59:30,  1.55s/it]

Checkpoint saved at index 2700


Processing articles:   8%|███████▍                                                                                 | 200/2400 [05:13<53:58,  1.47s/it]

Checkpoint saved at index 2800


...

Checkpoint saved at index 4800


Processing articles:  96%|████████████████████████████████████████████████████████████████████████████████████▎   | 2300/2400 [57:28<02:17,  1.38s/it]

Checkpoint saved at index 4900


Processing articles: 100%|████████████████████████████████████████████████████████████████████████████████████████| 2400/2400 [59:55<00:00,  1.50s/it]

Checkpoint saved at index 5000
Processing complete and final checkpoint saved.


In [36]:
import joblib
joblib.dump(responses, f"../data/nonethnic_{N}_gpt3.5_turbo_{EXPERIMENT_IDENTIFIER}_v3.joblib")

['../data/nonethnic_5000_gpt3.5_turbo_debug_v3.joblib']

In [37]:
joblib.dump(articles, f"../data/nonethnic_articles_{N}_gpt3.5_turbo_{EXPERIMENT_IDENTIFIER}_v3.joblib")

['../data/nonethnic_articles_5000_gpt3.5_turbo_debug_v3.joblib']

In [24]:
with open(checkpoint_file, 'rb') as f:
    responses = pickle.load(f)

In [38]:
len(responses)

5000

In [40]:
ethnic_sample_df = pd.DataFrame(articles)
ethnic_sample_df.columns = ["content"]
ethnic_sample_df

,content
0,<p>ফকিরহাট উপজেলা স্বাস্থ্য ও পরিবার পরিকল্পনা...
1,ব্লু-হোয়েল শেষে অতঃপর চলছে মোমো আতঙ্ক। তবে মোম...
2,"<p>এদিকে প্রযোজক জানিয়েছেন, ছবিটির জন্য প্রচার..."
3,"<p class=""Hdg1col"">প্রথমবারের মতো দক্ষিণ আফ্রি..."
4,<p>মফস্বলের শিক্ষার্থীদের ইংরেজি–ভীতি দূর করতে...
...,...
4995,জন্মের দেড় বছর পেরিয়ে গেলেও শরীরে অস্ত্রোপচার ...
4996,"<p>শিক্ষামন্ত্রী নুরুল ইসলাম নাহিদ বলেছেন, প্র..."
4997,"<p class=""TEXT"">দেশে করোনা শনাক্তকরণ পরীক্ষা শ..."
4998,বিএনপির ভারপ্রাপ্ত মহাসচিব মির্জা ফখরুল ইসলাম ...


In [72]:
import re

def fix_quotes(json_str):
    #print(f"Before fixing: {json_str}")
    apostrophe = '"'
    parts = json_str.split(",")
    fixed_parts = []
    for part in parts:
        if "genre" not in part:

            # Detecting error
            count_apostrophe = 0
            for alphabet in part:
                if alphabet == apostrophe:
                    count_apostrophe+=1
            # Fixing
            if count_apostrophe > 4:
                better_part = []
                count_apostrophe = 0
                for alphabet in part:
                    if alphabet == apostrophe:
                        count_apostrophe+=1
                    if count_apostrophe == 4:
                        better_part.append("'")
                        count_apostrophe = 0
                    else:
                        better_part.append(alphabet)
                part = "".join(better_part)
                
            fixed_parts.append(part)
        else:
            fixed_parts.append(part)
        
    temp = ", ".join(fixed_parts)
    #print(f"After fixing: {temp}")
    return temp
    
def post_process_chatgpt_turbo_3_5_response_to_json(text):
    json_string = text.replace("```", "").replace("json","").replace("\n","").replace("'", '"').replace("\t","").replace('"s ', '\'s ')
    try:
        json_output = json.loads(json_string)
        return json_output
    except:
        json_output = json.loads(fix_quotes(json_string))
        return json_output

In [75]:
ethnic_sample_df["Genre"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["genre"] for response in responses]
ethnic_sample_df["Journalism Style"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["journalism_style"] for response in responses]
ethnic_sample_df["Featured"] = [ post_process_chatgpt_turbo_3_5_response_to_json(response).get('featured', pd.NA) for response in responses ]
ethnic_sample_df["Is_ethnic"] = [post_process_chatgpt_turbo_3_5_response_to_json(response)["is_indigenous"] for response in responses]
ethnic_sample_df["Sentiment"] = [ post_process_chatgpt_turbo_3_5_response_to_json(response).get('sentiment', pd.NA) for response in responses ]
ethnic_sample_df["Agenda Setting"] = [ post_process_chatgpt_turbo_3_5_response_to_json(response).get('agenda_setting', pd.NA) for response in responses ]
ethnic_sample_df["Framing"] = [ post_process_chatgpt_turbo_3_5_response_to_json(response).get('framing', pd.NA) for response in responses ]
ethnic_sample_df["Priming"] = [ post_process_chatgpt_turbo_3_5_response_to_json(response).get('priming', pd.NA) for response in responses ]
ethnic_sample_df["Mobilizing"] = [ post_process_chatgpt_turbo_3_5_response_to_json(response).get('mobilizing', pd.NA) for response in responses ]

ethnic_sample_df

,content,Genre,Journalism Style,Is_ethnic,Featured,Sentiment,Agenda Setting,Framing,Priming,Mobilizing
0,<p>ফকিরহাট উপজেলা স্বাস্থ্য ও পরিবার পরিকল্পনা...,[Local news],Straight,False,Subject,Negative,Health,Tragedy,Awareness,Support
1,ব্লু-হোয়েল শেষে অতঃপর চলছে মোমো আতঙ্ক। তবে মোম...,[Crime],Straight,False,Subject,Negative,Online Games,Dangerous Influence,Youth,Awareness
2,"<p>এদিকে প্রযোজক জানিয়েছেন, ছবিটির জন্য প্রচার...",[Culture & Entertainment],Straight,False,Not applicable,Neutral,Promotion,Promotion,Advertisement,None
3,"<p class=""Hdg1col"">প্রথমবারের মতো দক্ষিণ আফ্রি...",[Sports],Straight,False,Subject,Positive,Sports,Competition,Victory,Celebration
4,<p>মফস্বলের শিক্ষার্থীদের ইংরেজি–ভীতি দূর করতে...,[Education],Straight,False,Subject,Negative,English Language,Competition,Students,Improvement
...,...,...,...,...,...,...,...,...,...,...
4995,জন্মের দেড় বছর পেরিয়ে গেলেও শরীরে অস্ত্রোপচার ...,"[Health, Local News]",Straight,False,Personal Experience,Positive,Treatment,Family Issues,Healthcare,Community Support
4996,"<p>শিক্ষামন্ত্রী নুরুল ইসলাম নাহিদ বলেছেন, প্র...","[Politics, Education]",Straight,False,Subject,Neutral,Education,Policy,Feedback,Reform
4997,"<p class=""TEXT"">দেশে করোনা শনাক্তকরণ পরীক্ষা শ...",[Health],Straight,False,Expert or Commentator,Neutral,Testing,Prevention,Healthcare,Awareness
4998,বিএনপির ভারপ্রাপ্ত মহাসচিব মির্জা ফখরুল ইসলাম ...,"[Politics, Crime]",Straight,False,Subject,Negative,Legal,Controversy,Court,Protest


In [76]:
ethnic_sample_df.isna().sum()

content               0
Genre                 0
Journalism Style     44
Is_ethnic             1
Featured            972
Sentiment            81
Agenda Setting       73
Framing              72
Priming              75
Mobilizing           78
dtype: int64

In [77]:
ethnic_sample_df.to_csv(f"../data/nonethnic_{N}_gpt_3.5_turbo_v3.csv")

### Evaluating

In [25]:
true_labels = [1,1,0,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,1,1,0,0,1,0,1,0,1,1,0,1,0,0,1,0,1,1,1,1,0,1,1,1,0,1,1,1,0,
               1,1,1,0,1,0,0,0,1,1,1,1,0,1,0,1,
               0,1,0,1,1,1,1,0,1,1,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,1,1,1,1,0,1,1,1,1,0,1]

In [27]:
predicted_labels = ethnic_sample_df["Is_ethnic"].values.tolist()
predicted_labels = [1 if item == True else 0 for item in predicted_labels]
predicted_labels[:10]

[1, 0, 1, 0, 1]

In [28]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

# Calculating precision, recall, accuracy, and F1 score
precision = precision_score(true_labels[:N], predicted_labels)
recall = recall_score(true_labels[:N], predicted_labels)
accuracy = accuracy_score(true_labels[:N], predicted_labels)
f1 = f1_score(true_labels[:N], predicted_labels)

# Print the results
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Precision: 0.6666666666666666
Recall: 0.6666666666666666
Accuracy: 0.6
F1 Score: 0.6666666666666666


In [108]:
ethnic_sample_df["Is_ethnic"].values.tolist()

[True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True]

### MISC

In [11]:
abhi_labels = [1,1,0,0,1,1,1,0,1,1,0,0,1,1,1,1,1,1,0,
               1,1,0,0,1,0,1,0,1,1,0,1,1,0,0,0,1,1,1,
               1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,0,1,1,
               1,1,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,
               0,0,1,0,1,
               1,0,0,0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,]
sharif_labels = [ 1,1,0,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,1,0,1,1,1,0,1,1,1,0,
                 1,1,1,0,1,0,0,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,1,1,0,1,1,1,1,0,1 ]

In [14]:
import numpy as np
def annotator_agreement_score(labels1, labels2):
    # Calculate agreement score
    total_labels = len(labels1)
    agreements = np.sum(np.array(labels1) == np.array(labels2))
    agreement_score = agreements / total_labels

    # Find conflicting posts
    conflict_indices = [i for i in range(total_labels) if labels1[i] != labels2[i]]
    conflicting_posts = [(i, labels1[i], labels2[i]) for i in conflict_indices]

    return agreement_score, len(conflicting_posts)
annotator_agreement_score(abhi_labels, sharif_labels)

(np.float64(0.75), 25)

In [80]:
from collections import Counter

counts = Counter(true_labels)
print("Yes count:", counts[1])
print("No count:", counts[0])

Yes count: 63
No count: 37


In [45]:
ethnic_sample_df.iloc[2].values.tolist()

[np.int64(2),
 'গাইবান্ধার গোবিন্দগঞ্জের রংপুর চিনিকলের সাহেবগঞ্জ ইক্ষু খামারে দেয়া কাঁটাতারের বেড়ার পিলার ভাংচুর করেছে দুর্বৃত্তরা।সোমবার রাতে খামার সংলগ্ন জয়পুর পাড়ার দক্ষিণ অংশে দেয়া ২৬টি পিলার ভাংচুর করা হয়।খামারের জমি সাঁওতালদের কাছ থেকে দখলমুক্ত করার পর মিল কর্তৃপক্ষ এই কাঁটাতারের বেড়া দেয়। এরপর সোমবার রাতে সাঁওতাল পল্লী সংলগ্ন খামারের বেড়ার পিলার ভাংচুর করে দুর্বৃত্তরা।রংপুর সুগার মিলের ব্যবস্থাপনা পরিচালক আব্দুল আউয়াল জানান, দুস্কৃতকারীরা রাতের আধারে খামারের পশ্চিম অংশে সাঁওতাল পল্লী জয়পুর পাড়া সংলগ্ন ২৬টি পিলার ভাংচুর করে। এ ঘটনায় ১৩ জনের নাম উল্লেখ করে খামারের উপ-ব্যবস্থাপক মো. হুমায়ন কবির মঙ্গলবার রাতে গোবিন্দগঞ্জ থানায় একটি জিডি করেন।গোবিন্দগঞ্জ থানার ওসি সুব্রত কুমার সরকার বিষয়টি নিশ্চিত করে বলেন, ঘটনার বিষয়টি খতিয়ে দেখা হচ্ছে।গত ৬ নভেম্বর আদিবাসী সাঁওতালদের সঙ্গে আইনশৃংখলা বাহিনী চিনিকলের শ্রমিক-কর্মচারীদের সংঘর্ষের পর ইক্ষু খামার থেকে সাঁওতাল বসতি উচ্ছেদ, লুটপাট, গুলিতে ৩ সাঁওতাল নিহত হওয়ার ঘটনায় সেখানে এখনও অস্থিরতা রয়েছে।',
 'Crime',
 'Investigative',
 'Victim',
 np.Tr

In [78]:
ethnic_sample_df = ethnic_df.sample(n=100, random_state=3)

In [79]:
ethnic_sample_df.to_csv("../data/annotated_100_ethnic_article_against_5_metrics.csv")